# Final Course Scheduling - Complete Solution

## Features:
- ✅ Tutorials get 2 rooms (preferably at same time)
- ✅ Lecturer unavailability working
- ✅ Penalty for 08:30 lectures (not tutorials)
- ✅ Beautiful visual output with HTML export

## Step 1: Load Parameters

In [26]:
import pulp
import pandas as pd
from IPython.display import HTML, display

# Load all parameters
%run load_parameters.py

Programs (P): ['CS_Y1', 'CS_Y2', 'DS_Y1', 'DS_Y2']
Courses (C): ['BCS1220', 'BCS1440', 'BCS1530', 'BCS2110', 'BCS2210', 'BCS2720', 'KEN1440', 'KEN1530', 'KEN1220', 'KEN2230', 'KEN2240', 'KEN2530']
Rooms (R): ['EDP150', 'C0.004', 'C0.008', 'C0.016', 'C0.020', 'C1.005', 'C1.015', 'C2.007']
Lecturers (L): ['Tom', 'Ottie', 'Filippe', 'Tony', 'Francessco', 'Tang', 'Marieke', 'Stefan', 'Yan', 'Ashish 10', 'Charis']

Student counts (n_p): {'CS_Y1': 220, 'CS_Y2': 165, 'DS_Y1': 180, 'DS_Y2': 218}

Courses per program (Courses_p): {'CS_Y1': ['BCS1220', 'BCS1440', 'BCS1530'], 'CS_Y2': ['BCS2110', 'BCS2210', 'BCS2720'], 'DS_Y1': ['KEN1220', 'KEN1440', 'KEN1530'], 'DS_Y2': ['KEN2230', 'KEN2240', 'KEN2530']}

Course timelines (Timeline_c): {'BCS1220': ['C', 'C', 'T'], 'BCS1440': ['C', 'C', 'T'], 'BCS1530': ['C', 'C', 'T'], 'BCS2110': ['C', 'C', 'T'], 'BCS2210': ['C', 'C', 'T'], 'BCS2720': ['C', 'C', 'T'], 'KEN1440': ['C', 'C', 'T'], 'KEN1530': ['C', 'C', 'T'], 'KEN1220': ['C', 'C', 'T'], 'KEN2230': 

In [27]:
params['U'] = {
    'Ottie': ['Monday'],
    'Tony': ['Friday'],
    'Tang': ['Monday'],
    'Marieke': ['Friday'],
    'Yan': ['Wednesday', 'Thursday'],
    'Ashish': ['Wednesday', 'Thursday'],
    'Ashish 10': ['Wednesday', 'Thursday'],
    'Charis': ['Wednesday', 'Thursday']
}

## Step 2: Build the Optimization Model

In [28]:
# Build the complete model
%run final_pulp_solution.py

model, x, y = create_optimized_model(params)


    Final Course Scheduling Model
    
    Usage in notebook:
    
    # Load parameters
    %run load_parameters.py
    
    # Create and solve model
    %run final_model.py
    model, x, y = create_optimized_model(params)
    status = model.solve(pulp.PULP_CBC_CMD(msg=1))
    
    print(f"Status: {pulp.LpStatus[status]}")
    
    # Visualize if successful
    if status == pulp.LpStatusOptimal:
        %run visualize_schedule.py
        schedule_df = display_all_views(x, params)
    

BUILDING OPTIMIZED COURSE SCHEDULING MODEL
✅ Created 5760 main decision variables
✅ Created 240 tutorial timing variables
✅ Objective function set: minimize early lectures + tutorial splits
✅ Constraint 1: Sessions scheduled (tutorials get 2 rooms)
✅ Linked tutorial timing variables
✅ Constraint 2: Room capacity requirements
✅ Constraint 3: No room double-booking
✅ Constraint 4: Lecturer unavailability (120 constraints)
✅ Constraint 5: Lecturer no double-booking
✅ Constraint 6: Student no overlap
⏳ Add

## Step 3: Solve the Model

In [29]:
print("\nSolving the model...")
print("This may take 1-2 minutes...\n")

status = model.solve(pulp.PULP_CBC_CMD(msg=1))

print("\n" + "="*80)
print(f"Solution Status: {pulp.LpStatus[status]}")
print("="*80)

if status == pulp.LpStatusOptimal:
    print("\n✅ SUCCESS! Found optimal schedule!")
    print(f"Objective value: {pulp.value(model.objective):.0f}")
    print("  (Lower is better - represents penalties for early lectures + split tutorials)")
elif status == pulp.LpStatusInfeasible:
    print("\n❌ INFEASIBLE: No valid schedule exists")
else:
    print(f"\n⚠️  Solver status: {pulp.LpStatus[status]}")


Solving the model...
This may take 1-2 minutes...

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/9c9fbfb9ab38407d830e242d0fb9bb01-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/9c9fbfb9ab38407d830e242d0fb9bb01-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 331101 COLUMNS
At line 1024702 RHS
At line 1355799 BOUNDS
At line 1361800 ENDATA
Problem MODEL has 331096 rows, 6000 columns and 680400 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 120 - 1.22 seconds
Cgl0002I 2760 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 23885 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 25503 strengthened rows, 0 substituti

## Step 4: Visualize the Schedule

In [30]:
if status == pulp.LpStatusOptimal:
    # Load visualization tools
    %run visualize_schedule.py
    
    # Display all views
    schedule_df = display_all_views(x, params)
else:
    print("Cannot visualize - model did not find a solution")


    Visual Schedule Display Tool
    
    Usage in Jupyter notebook:
    
    from visualize_schedule import display_all_views
    
    # After solving your model:
    df = display_all_views(x, clean_params)
    
    # Or use individual functions:
    from visualize_schedule import create_schedule_dataframe, create_html_calendar
    df = create_schedule_dataframe(x, clean_params)
    html = create_html_calendar(df, clean_params)
    display(HTML(html))
    

SCHEDULE SUMMARY
Total sessions scheduled: 48
Courses: 12
Rooms used: 4
Lecturers: 11

WEEKLY SCHEDULE

MONDAY

08:30-10:30:
  • KEN2240 - Session 0 (Lecture)
    Room: C0.020 (capacity 148)
    Lecturer: Ashish
    Students: 218

11:00-13:00:
  • BCS2210 - Session 0 (Lecture)
    Room: C0.020 (capacity 148)
    Lecturer: Francessco
    Students: 165
  • KEN1440 - Session 0 (Lecture)
    Room: C0.004 (capacity 120)
    Lecturer: Marieke
    Students: 180
  • KEN2530 - Session 0 (Lecture)
    Room: C0.016 (capacity 128)
    Lecture

Time,Monday,Tuesday,Wednesday,Thursday,Friday
08:30-10:30,KEN2240 - Session 0 Lecture | Room C0.020 218 students Ashish,BCS2210 - Session 2 Tutorial | Room C0.004 165 students Francessco KEN2530 - Session 2 Tutorial | Room EDP150 218 students Charis,KEN1440 - Session 2 Tutorial | Room C0.020 180 students Marieke,BCS1530 - Session 2 Tutorial | Room C0.016 220 students Filippe KEN1220 - Session 2 Tutorial | Room C0.020 180 students Tom,BCS1440 - Session 2 Tutorial | Room C0.020 220 students Ottie KEN1220 - Session 2 Tutorial | Room EDP150 180 students Tom KEN2240 - Session 2 Tutorial | Room C0.016 218 students Ashish
11:00-13:00,BCS2210 - Session 0 Lecture | Room C0.020 165 students Francessco KEN1440 - Session 0 Lecture | Room C0.004 180 students Marieke KEN2530 - Session 0 Lecture | Room C0.016 218 students Charis,KEN1220 - Session 1 Lecture | Room EDP150 180 students Tom KEN2240 - Session 1 Lecture | Room C0.016 218 students Ashish,BCS2720 - Session 0 Lecture | Room C0.020 165 students Tang,BCS1440 - Session 0 Lecture | Room C0.020 220 students Ottie,BCS1440 - Session 2 Tutorial | Room C0.004 220 students Ottie BCS2720 - Session 2 Tutorial | Room C0.016 165 students Tang KEN2230 - Session 1 Lecture | Room EDP150 218 students Yan
13:30-15:30,BCS2210 - Session 1 Lecture | Room C0.016 165 students Francessco KEN1530 - Session 0 Lecture | Room C0.004 180 students Stefan KEN2530 - Session 1 Lecture | Room C0.020 218 students Charis,BCS1530 - Session 1 Lecture | Room EDP150 220 students Filippe KEN1440 - Session 1 Lecture | Room C0.004 180 students Marieke KEN2240 - Session 2 Tutorial | Room C0.016 218 students Ashish,BCS1530 - Session 2 Tutorial | Room C0.004 220 students Filippe BCS2110 - Session 1 Lecture | Room EDP150 165 students Tony KEN1440 - Session 2 Tutorial | Room C0.020 180 students Marieke,BCS1220 - Session 1 Lecture | Room C0.016 220 students Tom BCS2110 - Session 2 Tutorial | Room C0.004 165 students Tony KEN1530 - Session 2 Tutorial | Room EDP150 180 students Stefan,BCS1220 - Session 2 Tutorial | Room C0.020 220 students Tom BCS2720 - Session 2 Tutorial | Room C0.004 165 students Tang KEN2230 - Session 2 Tutorial | Room EDP150 218 students Yan
16:00-18:00,BCS1530 - Session 0 Lecture | Room C0.020 220 students Filippe KEN1220 - Session 0 Lecture | Room C0.004 180 students Tom KEN2530 - Session 2 Tutorial | Room C0.016 218 students Charis,BCS2110 - Session 0 Lecture | Room C0.020 165 students Tony KEN1530 - Session 1 Lecture | Room C0.016 180 students Stefan KEN2230 - Session 0 Lecture | Room EDP150 218 students Yan,BCS1220 - Session 0 Lecture | Room C0.016 220 students Tom BCS2720 - Session 1 Lecture | Room C0.020 165 students Tang,BCS1440 - Session 1 Lecture | Room C0.004 220 students Ottie BCS2110 - Session 2 Tutorial | Room C0.020 165 students Tony KEN1530 - Session 2 Tutorial | Room EDP150 180 students Stefan,BCS1220 - Session 2 Tutorial | Room C0.004 220 students Tom BCS2210 - Session 2 Tutorial | Room C0.020 165 students Francessco KEN2230 - Session 2 Tutorial | Room EDP150 218 students Yan


## Step 5: Check Tutorial Assignments

In [31]:
if status == pulp.LpStatusOptimal and schedule_df is not None:
    print("\n" + "="*80)
    print("TUTORIAL ROOM ASSIGNMENTS")
    print("="*80)
    
    tutorials = schedule_df[schedule_df['Type'] == 'T'].sort_values(['Course', 'Session', 'Day_Num', 'Time_Num'])
    
    for course in tutorials['Course'].unique():
        course_tutorials = tutorials[tutorials['Course'] == course]
        
        print(f"\n{course}:")
        
        for session in course_tutorials['Session'].unique():
            session_rooms = course_tutorials[course_tutorials['Session'] == session]
            
            if len(session_rooms) == 2:
                room1 = session_rooms.iloc[0]
                room2 = session_rooms.iloc[1]
                
                if (room1['Day'] == room2['Day'] and room1['Time'] == room2['Time']):
                    print(f"  ✅ Session {session}: SAME TIME")
                    print(f"     {room1['Day']} {room1['Time']}")
                    print(f"     Rooms: {room1['Room']} + {room2['Room']}")
                else:
                    print(f"  ⚠️  Session {session}: DIFFERENT TIMES")
                    print(f"     Room 1: {room1['Day']} {room1['Time']} in {room1['Room']}")
                    print(f"     Room 2: {room2['Day']} {room2['Time']} in {room2['Room']}")
            else:
                print(f"  ❌ Session {session}: ERROR - has {len(session_rooms)} rooms (should be 2)")


TUTORIAL ROOM ASSIGNMENTS

BCS1220:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Friday 13:30-15:30 in C0.020
     Room 2: Friday 16:00-18:00 in C0.004

BCS1440:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Friday 08:30-10:30 in C0.020
     Room 2: Friday 11:00-13:00 in C0.004

BCS1530:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Wednesday 13:30-15:30 in C0.004
     Room 2: Thursday 08:30-10:30 in C0.016

BCS2110:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Thursday 13:30-15:30 in C0.004
     Room 2: Thursday 16:00-18:00 in C0.020

BCS2210:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Tuesday 08:30-10:30 in C0.004
     Room 2: Friday 16:00-18:00 in C0.020

BCS2720:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Friday 11:00-13:00 in C0.016
     Room 2: Friday 13:30-15:30 in C0.004

KEN1220:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Thursday 08:30-10:30 in C0.020
     Room 2: Friday 08:30-10:30 in EDP150

KEN1440:
  ⚠️  Session 2: DIFFERENT TIMES
     Room 1: Wednesday 08:30-

## Step 6: Check Early Morning Penalties

In [32]:
if status == pulp.LpStatusOptimal and schedule_df is not None:
    print("\n" + "="*80)
    print("EARLY MORNING (08:30) ANALYSIS")
    print("="*80)
    
    early_sessions = schedule_df[schedule_df['Time'] == '08:30-10:30'].sort_values(['Day', 'Course'])
    
    lectures_830 = early_sessions[early_sessions['Type'] == 'C']
    tutorials_830 = early_sessions[early_sessions['Type'] == 'T']
    
    print(f"\nLectures at 08:30: {len(lectures_830)} (penalized)")
    for _, session in lectures_830.iterrows():
        print(f"  📚 {session['Day']:10} | {session['Course']:10} Session {session['Session']} | Room {session['Room']}")
    
    print(f"\nTutorials at 08:30: {len(tutorials_830)} (NOT penalized)")
    for _, session in tutorials_830.iterrows():
        print(f"  ✏️  {session['Day']:10} | {session['Course']:10} Session {session['Session']} | Room {session['Room']}")
    
    total_penalty = len(lectures_830) * 100
    print(f"\nTotal early morning penalty: {total_penalty}")


EARLY MORNING (08:30) ANALYSIS

Lectures at 08:30: 1 (penalized)
  📚 Monday     | KEN2240    Session 0 | Room C0.020

Tutorials at 08:30: 8 (NOT penalized)
  ✏️  Friday     | BCS1440    Session 2 | Room C0.020
  ✏️  Friday     | KEN1220    Session 2 | Room EDP150
  ✏️  Friday     | KEN2240    Session 2 | Room C0.016
  ✏️  Thursday   | BCS1530    Session 2 | Room C0.016
  ✏️  Thursday   | KEN1220    Session 2 | Room C0.020
  ✏️  Tuesday    | BCS2210    Session 2 | Room C0.004
  ✏️  Tuesday    | KEN2530    Session 2 | Room EDP150
  ✏️  Wednesday  | KEN1440    Session 2 | Room C0.020

Total early morning penalty: 100


## Step 7: COMPREHENSIVE VERIFICATION - All 7 Constraints

In [33]:
if status == pulp.LpStatusOptimal and schedule_df is not None:
    # Run comprehensive verification of all constraints
    %run verify_all_constraints.py
    all_passed = verify_all_constraints(schedule_df, params)


    Comprehensive Constraint Verification
    
    Usage in notebook:
    
    %run verify_all_constraints.py
    all_passed = verify_all_constraints(schedule_df, params)
    

COMPREHENSIVE CONSTRAINT VERIFICATION

CONSTRAINT 1: Session Scheduling
✅ PASSED
   Lectures: 24 sessions (1 room each)
   Tutorials: 24 sessions (should be pairs, 2 rooms each)

CONSTRAINT 2: Room Capacity
✅ PASSED
   All 48 sessions have sufficient room capacity

CONSTRAINT 3: No Room Double-Booking
✅ PASSED
   No room conflicts found across 48 sessions

CONSTRAINT 4: Lecturer Unavailability
✅ PASSED
   All 8 lecturers with unavailability are respected

CONSTRAINT 5: Lecturer No Double-Booking
✅ PASSED
   No lecturer conflicts across 11 lecturers

CONSTRAINT 6: Student No Overlap
✅ PASSED
   No student conflicts across 4 programs

CONSTRAINT 7: Chronological Ordering
✅ PASSED
   All 12 courses have sessions in correct chronological order

VERIFICATION SUMMARY

🎉 ALL CONSTRAINTS SATISFIED! 🎉
   The schedule is

## Step 8: Export to HTML File

In [36]:
if status == pulp.LpStatusOptimal and schedule_df is not None:
    from visualize_schedule import create_html_calendar
    
    # Create the HTML
    html_content = create_html_calendar(schedule_df, params)
    
    # Add some styling and header
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Course Schedule - Week of Oct 27, 2025</title>
    </head>
    <body>
        <h1 style="text-align: center; color: #2c3e50;">📅 Course Schedule</h1>
        <p style="text-align: center; color: #666;">Week of October 27-31, 2025</p>
        {html_content}
        
        <div style="margin: 40px; padding: 20px; background: #f8f9fa; border-radius: 10px;">
            <h3>📊 Summary Statistics</h3>
            <p><strong>Total Sessions:</strong> {len(schedule_df)}</p>
            <p><strong>Courses:</strong> {schedule_df['Course'].nunique()}</p>
            <p><strong>Rooms Used:</strong> {schedule_df['Room'].nunique()}</p>
            <p><strong>Lecturers:</strong> {schedule_df['Lecturer'].nunique()}</p>
            <p><strong>Early Morning Lectures (08:30):</strong> {len(schedule_df[(schedule_df['Time'] == '08:30-10:30') & (schedule_df['Type'] == 'C')])}</p>
        </div>
    </body>
    </html>
    """
    
    # Save to file
    with open('course_schedule.html', 'w', encoding='utf-8') as f:
        f.write(full_html)
    
    print("✅ Schedule exported to 'course_schedule.html'")
    print("   You can open this file in any web browser!")
    print("\n   On Mac: Open Finder, double-click the file")
    print("   Or run: !open course_schedule.html")

✅ Schedule exported to 'course_schedule.html'
   You can open this file in any web browser!

   On Mac: Open Finder, double-click the file
   Or run: !open course_schedule.html


## Step 9: Export to Excel

In [ ]:
if status == pulp.LpStatusOptimal and schedule_df is not None:
    # Export to Excel
    schedule_df.to_excel('course_schedule.xlsx', index=False)
    print("✅ Schedule also exported to 'course_schedule.xlsx'")